In [3]:
from luxonis_ml.data.dataset import LuxonisDataset
from luxonis_ml.data.parsers import LuxonisParser, DatasetType

## COCO example

In [2]:
# Initialize the LuxonisParser
parser = LuxonisParser()

In [1]:
# !wget http://images.cocodataset.org/zips/val2017.zip
# !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

# # Unzip the downloaded files
# !unzip val2017.zip
# !unzip annotations_trainval2017.zip

--2023-10-02 08:17:29--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.24.123, 3.5.24.139, 3.5.29.133, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.24.123|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80M  80.4MB/s    in 8.7s    

2023-10-02 08:17:38 (89.0 MB/s) - ‘val2017.zip’ saved [815585330/815585330]

--2023-10-02 08:17:39--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.211.161, 52.217.173.153, 52.217.159.1, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.211.161|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[==

In [4]:
image_dir = "val2017"  # Directory containing the images
annotation_path = "annotations/instances_val2017.json"  # Path to COCO annotations
split = "val"  # Data split ('train', 'val', or 'test')

In [5]:

# Initialize LuxonisDataset 
dataset_name = "coco_parser_test"
# team_name = "luxonis_name"
# team_id = "luxonis_id"

# Using LuxonisDataset.create to create a new dataset or get the ID of an existing one
dataset_id = LuxonisDataset.create(dataset_name=dataset_name)


In [6]:
dataset_type = DatasetType.COCO
# dataset_type = DatasetType.S3DIR
dataset_info = (dataset_name, dataset_type) # dataset_name, and dataset_type

In [7]:
# Parse the dataset
parser.from_coco_format(dataset_info, image_dir, annotation_path, split=split)

setting component
setting source
started parsing..


100%|██████████| 5000/5000 [01:02<00:00, 79.41it/s]
2023-09-27 14:28:53,273 [INFO] Checking for additions or modifications...
100%|██████████| 36781/36781 [00:43<00:00, 836.65it/s]
2023-09-27 14:29:37,239 [INFO] Extracting dataset media...
100%|██████████| 36781/36781 [19:20<00:00, 31.70it/s] 
2023-09-27 14:48:57,361 [INFO] Executing changes to dataset...
100%|██████████| 36782/36782 [00:41<00:00, 876.33it/s]
2023-09-27 14:49:39,732 [INFO] Adding samples to Fiftyone


 100% |█████████████| 36781/36781 [3.1m elapsed, 0s remaining, 196.0 samples/s]      


2023-09-27 14:52:44,765 [INFO]  100% |█████████████| 36781/36781 [3.1m elapsed, 0s remaining, 196.0 samples/s]      


In [8]:
with LuxonisDataset(    
    dataset_name=dataset_name
) as dataset:
    
    for sample in dataset.fo_dataset:
        print(sample)
        break

<Sample: {
    'id': '651441034488dfee5202d837',
    'media_type': 'image',
    'filepath': '/offline/datasets/65143be54488dfee52000b90/image/6377ebe5-6f85-5181-b7c9-13f88df95956.jpg',
    'tags': [],
    'metadata': None,
    'image': <Group: {'id': '65143c254488dfee52000b93', 'name': 'image'}>,
    'version': -1.0,
    'latest': False,
    'annotated': True,
    'tid': '65143c254488dfee52000b94',
    'created_at': datetime.datetime(2023, 9, 27, 14, 48, 57, 759000),
    'split': 'val',
    'class': <Classifications: {
        'classifications': [
            <Classification: {
                'id': '651440d94488dfee52012aee',
                'tags': [],
                'label': 'bottle',
                'confidence': None,
                'logits': None,
            }>,
        ],
        'logits': None,
    }>,
    'segmentation': <Segmentation: {
        'id': '651440d94488dfee52012aef',
        'tags': [],
        'mask': array([[0, 0, 0, ..., 0, 0, 0],
               [0, 0, 0, ...

In [ ]:
# with LuxonisDataset(dataset_name) as dataset:
#     dataset.delete_dataset()

2023-09-26 08:35:58,296 [WARNING] Dataset persistence on exit failed for reason Dataset 'offline-65119abf1afef4af576acca5' is deleted


## YOLO example

In [1]:
# Clone the yolov4-custom_Training repository, for class names, train/test split, and yolov4 config
!git clone https://github.com/techzizou/yolov4-custom_Training.git

# Move train.txt and test.txt to yolov4-custom_Training/yolov4/
!mv yolov4-custom_Training/yolov4/generated_text_files/train.txt yolov4-custom_Training/yolov4/train.txt
!mv yolov4-custom_Training/yolov4/generated_text_files/test.txt yolov4-custom_Training/yolov4/test.txt

# get the dataset (images and labels) from kaggle
%mkdir yolov4-custom_Training/yolov4/data
%cd yolov4-custom_Training/yolov4/data
!kaggle datasets download -d techzizou/labeled-mask-dataset-yolo-darknet
!unzip labeled-mask-dataset-yolo-darknet.zip
!rm labeled-mask-dataset-yolo-darknet.zip

# Remove spaces and brackets from file names
%cd obj/
!rename 's/ /_/g' *
!rename 's/\(|\)//g' *
%cd ../../

# Remove spaces and brackets from paths in train.txt and test.txt
!sed -i -e 's/ /_/g' -e 's/[(|)]//g' train.txt
!sed -i -e 's/ /_/g' -e 's/[(|)]//g' test.txt

%cd ../..

Cloning into 'yolov4-custom_Training'...
remote: Enumerating objects: 299, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 299 (delta 61), reused 149 (delta 56), pack-reused 144
Receiving objects: 100% (299/299), 33.33 MiB | 71.25 MiB/s, done.
Resolving deltas: 100% (129/129), done.
/home/paperspace/Luxonis/luxonis-ml/examples/yolov4-custom_Training/yolov4/data
 88%|████████████████████████████████████▏    | 111M/126M [00:00<00:00, 128MB/s]
100%|█████████████████████████████████████████| 126M/126M [00:00<00:00, 135MB/s]
Archive:  labeled-mask-dataset-yolo-darknet.zip
  inflating: obj/0-with-mask.jpg     
  inflating: obj/0-with-mask.txt     
  inflating: obj/0.jpg               
  inflating: obj/0.txt               
  inflating: obj/1-with-mask.jpg     
  inflating: obj/1-with-mask.txt     
  inflating: obj/10-with-mask.jpg    
  inflating: obj/10-with-mask.txt    
  inflating: obj/100-with-mask.jpg   
  inflating: ob

In [2]:
from luxonis_ml.data.dataset import LuxonisDataset
from luxonis_ml.data.parsers import LuxonisParser, DatasetType
from luxonis_ml.data.dataset_recognition import recognize

In [3]:
dataset_type = recognize('yolov4-custom_Training/yolov4/')

In [4]:
# Initialize the LuxonisParser
parser = LuxonisParser()

In [5]:
# Initialize LuxonisDataset 
dataset_name = "yolov4_parser_test22"
# The following lines can be uncommented if you have team_name and team_id
# team_name = "luxonis_name"
# team_id = "luxonis_id"

# Using LuxonisDataset.create to create a new dataset or get the ID of an existing one
dataset_id = LuxonisDataset.create(dataset_name=dataset_name)


In [6]:
# dataset_type = DatasetType.YOLOv4  # Setting the dataset type to YOLOv4
dataset_type = DatasetType[dataset_type] # convert string to enum
dataset_info = (dataset_name, dataset_type)  # dataset_name and dataset_type


In [7]:
# get current working directory
import os
cwd = os.getcwd()

image_dir = '/yolov4-custom_Training/yolov4/'  # Directory containing the images
# image_dir is actually 'yolov4-custom_Training/yolov4/data/obj' but the paths in train.txt already have 'data/obj/' in them
annotations_dir = image_dir  # Directory containing the annotations
img_select_file_path = '/yolov4-custom_Training/yolov4/train.txt'  # Path to YOLO annotations
classes_txt_file_path = '/yolov4-custom_Training/yolov4/obj.names'  # Path listing the class names
split = "train"  # Data split ('train', 'val', or 'test')

In [8]:
# # optimal/usual case
# image_dir = 'dataset/images/'
# annotations_dir = 'dataset/labels/'
# img_select_file_path = 'dataset/train.txt'
# classes_txt_file_path = 'dataset/classes.txt'

In [9]:
# Parse the dataset
parser.from_yolo4_format(dataset_info, 
                         cwd+image_dir, 
                         cwd+annotations_dir,
                         cwd+img_select_file_path,
                         cwd+classes_txt_file_path,
                         split=split)

setting component
setting source
started parsing..


  4%|▍         | 55/1359 [00:00<00:02, 546.04it/s]

100%|██████████| 1359/1359 [00:07<00:00, 172.02it/s]
2023-10-02 17:45:52,275 [INFO] Checking for additions or modifications...
100%|██████████| 1359/1359 [00:01<00:00, 1155.79it/s]
2023-10-02 17:45:53,454 [INFO] Extracting dataset media...
100%|██████████| 1359/1359 [00:17<00:00, 78.53it/s]
2023-10-02 17:46:10,762 [INFO] Executing changes to dataset...
100%|██████████| 1360/1360 [00:01<00:00, 1028.15it/s]
2023-10-02 17:46:12,132 [INFO] Adding samples to Fiftyone


 100% |███████████████| 1359/1359 [1.4s elapsed, 0s remaining, 993.8 samples/s]        


2023-10-02 17:46:13,504 [INFO]  100% |███████████████| 1359/1359 [1.4s elapsed, 0s remaining, 993.8 samples/s]        


In [10]:
with LuxonisDataset(    
    dataset_name=dataset_name
) as dataset:
    
    for sample in dataset.fo_dataset:
        print(sample)
        break

<Sample: {
    'id': '651b01e4ac426ebefb3b3c53',
    'media_type': 'image',
    'filepath': '/offline/datasets/651b01c8ac426ebefb3b2035/image/a98dd3e1-384b-564e-8ff9-9a38a4408373.jpg',
    'tags': [],
    'metadata': None,
    'image': <Group: {'id': '651b01d0ac426ebefb3b2038', 'name': 'image'}>,
    'version': -1.0,
    'latest': False,
    'annotated': True,
    'tid': '651b01d0ac426ebefb3b2039',
    'created_at': datetime.datetime(2023, 10, 2, 17, 46, 10, 809000),
    'split': 'train',
    'class': <Classifications: {
        'classifications': [
            <Classification: {
                'id': '651b01e2ac426ebefb3b2ad7',
                'tags': [],
                'label': 'with_mask',
                'confidence': None,
                'logits': None,
            }>,
        ],
        'logits': None,
    }>,
    'boxes': <Detections: {
        'detections': [
            <Detection: {
                'id': '651b01e2ac426ebefb3b2ad8',
                'attributes': {},
        